<a href="https://www.kaggle.com/code/srbharathee/sg-helper-bot?scriptVersionId=235087622" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# SG Helper Bot: Multilingual, Search-Grounded Singapore Assistant

---

## Introduction

This is built as a capstone project for Gen AI Intensive Course Capstone 2025Q1 (https://www.kaggle.com/competitions/gen-ai-intensive-course-capstone-2025q1) . The concepts used are from the learning resorces shared during -day Gen AI Intensive Course with Google (https://rsvp.withgoogle.com/events/google-generative-ai-intensive_2025q1)  
  
This notebook demonstrates a **Singapore Helper Bot**—a multilingual AI assistant powered by Google Gemini.  
It can answer queries in English, Chinese, Malay, and Tamil, and provides up-to-date, trustworthy information by grounding its answers in real-time Google Search results.  
The bot is designed for Singaporeans and visitors seeking local, practical, and official information, including transport, food, healthcare, and government services.

**Key Features:**
- Multilingual Q&A (all official SG languages)
- Real-time Google Search grounding with citations
- Localized answers for Singapore context
- Ready for extension with Singapore government open data

---

## Approach

- **Foundation Model:** Uses Gemini (Flash or Pro experimental if available) for LLM-powered Q&A and translation.
- **Grounding:** Gemini’s built-in Google Search grounding ensures answers are up-to-date and cite sources.
- **Multilingual:** Accepts and responds in any of Singapore’s official languages, using prompt engineering and optional language detection.


---

## 1. Environment and Package Setup



In [1]:
!pip show google-genai


Name: google-genai
Version: 0.8.0
Summary: GenAI Python SDK
Home-page: https://github.com/googleapis/python-genai
Author: 
Author-email: Google LLC <googleapis-packages@google.com>
License: Apache-2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: google-auth, pydantic, requests, websockets
Required-by: 



**We use `google-generativeai` for responses without grounding and google-genai for results with google search grounding . This is to try the different modules available .An exhaustive comparison of the modules has not been done yet**

In [2]:
#!pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
#!pip install -U -q "google-genai==1.7.0"
!pip install -q google-generativeai

## 2. Obtain Google API key 

In [3]:
import os
from kaggle_secrets import UserSecretsClient
from IPython.display import Markdown, HTML, display

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [4]:
import google.generativeai as genai
genai.configure()  # This will pick up the API key from the environment

#or do 
'''genai.configure(
    api_key=GOOGLE_API_KEY,
    
)'''

'genai.configure(\n    api_key=GOOGLE_API_KEY,\n    \n)'


**This cell prints all available Gemini models.  
You can choose the latest or most suitable one for your use case.**


In [5]:
#if we wanted to list the models to make a selection . commented out for now
#for m in genai.list_models():
#    print(m)

## 3. Define Multilingual Helper Function



In [6]:
from google.generativeai import GenerativeModel
model = GenerativeModel('gemini-2.5-pro-exp-03-25')




def multilingual_helper(user_input, target_language=None):
    if target_language:
        prompt = f"Translate the following and answer in {target_language}: {user_input}"
    else:
        prompt = f"Detect the language and answer in the same language: {user_input}"
    response = model.generate_content(prompt)
    return response.text

# Usage example
print(multilingual_helper("请问附近哪里有素食餐厅？"))  # Chinese input
print(multilingual_helper("Where can I find vegan food near Bugis?", target_language="Malay"))


您好！检测到您使用的是**中文**。

要查找附近的素食餐厅，我建议您使用手机上的地图应用（例如百度地图、高德地图、谷歌地图等），然后直接搜索“素食餐厅”或“素食”。这样通常能最快、最准确地找到您当前位置周边的选项，并能看到评价、距离和导航信息。

因为我不知道您具体在哪里，所以无法直接推荐某个餐厅。希望您能顺利找到合意的素食餐厅！
Tentu, ini terjemahannya dan jawapan dalam Bahasa Melayu:

**Terjemahan:**
Di mana saya boleh cari makanan vegan berdekatan Bugis?

**Jawapan:**
Anda boleh mencari makanan vegan di beberapa tempat berdekatan Bugis:

1.  **Fortune Centre:** Bangunan ini sangat terkenal dengan pilihan makanan vegetarian dan vegan. Terdapat beberapa restoran dan gerai di sini, kebanyakannya menawarkan hidangan berasaskan tumbuhan. Anda boleh bertanya terus untuk memastikan hidangan tersebut adalah vegan (tanpa telur, tenusu, atau produk haiwan lain). Contoh restoran di sini termasuk Herbivore Japanese Vegetarian.
2.  **Restoran Vegetarian Berdekatan:** Terdapat beberapa restoran vegetarian Cina atau India di sekitar kawasan Bugis yang mungkin mempunyai pilihan vegan atau boleh menyediakan hidangan vegan atas permintaan.
3.  **Pusat Membeli-

## Few Shot examples 

In [7]:
few_shot_examples = """
Example 1:
Q: Where can I find vegan food near Bugis?
A: You can find vegan food at Fortune Centre, which has several vegan and vegetarian restaurants. Other options near Bugis include Loving Hut and Herbivore.

Example 2:
Q: 如何从裕廊东去牛车水？
A: 您可以搭乘地铁东西线（绿色线）从裕廊东站直接到牛车水站，中途在欧南园站换乘地铁东北线（紫色线）。

Example 3:
Q: Di mana saya boleh dapatkan makanan India halal di Yishun?
A: Anda boleh cuba restoran seperti Al-Azhar Eating Restaurant di Yishun atau kedai-kedai mamak di sekitar kawasan tersebut.
"""


In [8]:
def multilingual_helper_few_shot(user_input):
    prompt = f"""{few_shot_examples}

Now, answer the following question in the same style as above:
Q: {user_input}
A:"""
    response = model.generate_content(prompt)
    return response.text


In [9]:
print(multilingual_helper_few_shot("What are the CPF withdrawal rules at age 55?"))
print(multilingual_helper_few_shot("请问从三巴旺怎么去樟宜机场？"))


A: From age 55, you can withdraw up to $5,000 from your CPF savings unconditionally. You can also withdraw savings above the Full Retirement Sum (FRS) that needs to be set aside in your Retirement Account. The savings in your Retirement Account will provide you with monthly payouts during retirement.
A: 您可以从三巴旺地铁站搭乘南北线（红色线），在市政厅站或莱佛士坊站换乘东西线（绿色线），然后在丹那美拉站换乘前往樟宜机场的列车。


## 4. Install Language detection and add language detection capability

**While we could use Google genai capability to detect the language, detecting the language offline /locally is much more cost effective and efficient**

In [10]:
!pip install -q langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [11]:
from langdetect import detect

def detect_and_respond(user_input):
    lang = detect(user_input)
    return multilingual_helper(user_input, target_language=lang)


## Interactive User query input loop . commented out for submission 

In [12]:
#commented out to keep this reference run during submission
'''while True:
    user_query = input("Ask me anything (type 'quit' to exit): ")
    if user_query.lower() == 'quit':
        break
    print("Bot:", detect_and_respond(user_query))'''

'while True:\n    user_query = input("Ask me anything (type \'quit\' to exit): ")\n    if user_query.lower() == \'quit\':\n        break\n    print("Bot:", detect_and_respond(user_query))'

## 5. Sample queries with multilingual response in all of Singapore official languages 

In [13]:
user_query="Can you recommend laptop service centers around Kembangan to fix a Dell Laptop screen. Provide answers in Malay and English "
from IPython.display import Markdown, HTML, display
response_text= detect_and_respond(user_query)
display(Markdown(f"**Sg Helper Bot:** {response_text}"))


**Sg Helper Bot:** Okay, here are recommendations for Dell laptop screen repair service centers near Kembangan, provided in both English and Malay.

---

**English:**

Here are some options for getting your Dell laptop screen fixed near Kembangan:

1.  **Official Dell Support (Recommended, especially if under warranty):**
    *   **Check Warranty First:** Visit the Dell Singapore support website (www.dell.com/support/sg) and enter your laptop's service tag to check its warranty status.
    *   **Contact Dell Support:** Even if out of warranty, contacting Dell Support directly via their website or hotline is a good starting point. They can provide official repair options, costs, and direct you to their authorized service partners. This ensures genuine parts are used, although it might sometimes be more expensive or take longer if you need to send the device in.

2.  **Third-Party Laptop Repair Centers (Near Kembangan/East Coast Area):**
    *   While Kembangan itself might have limited specialized laptop repair shops, nearby areas like Bedok, Eunos, or Paya Lebar often have several options.
    *   **How to Find:** Search on Google Maps for "laptop repair Kembangan", "laptop screen repair Bedok", or "computer service Eunos". Look for shops with good reviews that explicitly mention screen replacement services.
    *   **Examples (You'll need to search for current specific shops and call them):** Look for established computer repair shops in Bedok Central, near Eunos MRT, or Paya Lebar Square/SingPost Centre areas.
    *   **Important:** Before visiting any third-party center:
        *   **Call Ahead:** Confirm they can repair screens specifically for your Dell laptop model.
        *   **Get a Quote:** Ask for an estimated cost for the screen replacement.
        *   **Ask About Warranty:** Inquire about any warranty they offer on the repair work and the replacement screen.
        *   **Check Turnaround Time:** Find out how long the repair is likely to take.

**Recommendation:**

*   Always check your Dell warranty first.
*   If out of warranty, contact Dell for an official quote, then compare it with quotes from 1-2 reputable third-party repair shops near you before deciding.

---

**Malay:**

Baiklah, berikut adalah beberapa cadangan untuk pusat servis pembaikan skrin komputer riba Dell berdekatan Kembangan:

1.  **Sokongan Rasmi Dell (Disyorkan, terutamanya jika masih dalam waranti):**
    *   **Semak Waranti Dahulu:** Layari laman web sokongan Dell Singapura (www.dell.com/support/sg) dan masukkan 'service tag' komputer riba anda untuk menyemak status waranti.
    *   **Hubungi Sokongan Dell:** Walaupun sudah tamat waranti, menghubungi Sokongan Dell secara terus melalui laman web atau talian hotline mereka adalah permulaan yang baik. Mereka boleh memberikan pilihan pembaikan rasmi, kos, dan mengarahkan anda ke rakan kongsi servis sah mereka. Ini memastikan alat ganti tulen digunakan, walaupun ia kadangkala mungkin lebih mahal atau mengambil masa lebih lama jika anda perlu menghantar peranti tersebut.

2.  **Pusat Servis Komputer Riba Pihak Ketiga (Berdekatan Kembangan/Kawasan East Coast):**
    *   Walaupun Kembangan sendiri mungkin mempunyai kedai pembaikan komputer riba khusus yang terhad, kawasan berdekatan seperti Bedok, Eunos, atau Paya Lebar selalunya mempunyai beberapa pilihan.
    *   **Cara Mencari:** Cari di Google Maps untuk "baiki laptop Kembangan", "baiki skrin laptop Bedok", atau "servis komputer Eunos". Cari kedai yang mempunyai ulasan yang baik dan menyatakan secara jelas perkhidmatan penggantian skrin.
    *   **Contoh (Anda perlu mencari kedai spesifik terkini dan menghubungi mereka):** Cari kedai pembaikan komputer yang sudah lama bertapak di Bedok Central, berdekatan MRT Eunos, atau kawasan Paya Lebar Square/SingPost Centre.
    *   **Penting:** Sebelum mengunjungi mana-mana pusat pihak ketiga:
        *   **Telefon Dahulu:** Sahkan mereka boleh membaiki skrin khusus untuk model komputer riba Dell anda.
        *   **Dapatkan Sebut Harga:** Tanya anggaran kos untuk penggantian skrin.
        *   **Tanya Tentang Waranti:** Bertanya tentang sebarang waranti yang mereka tawarkan ke atas kerja pembaikan dan skrin gantian.
        *   **Semak Tempoh Masa:** Ketahui berapa lama masa pembaikan dijangka akan diambil.

**Cadangan:**

*   Sentiasa semak waranti Dell anda terlebih dahulu.
*   Jika sudah tamat waranti, hubungi Dell untuk mendapatkan sebut harga rasmi, kemudian bandingkan dengan sebut harga dari 1-2 kedai pembaikan pihak ketiga yang bereputasi berdekatan anda sebelum membuat keputusan.

In [14]:
user_query="Get me a trend of the housing prices in Singapore. Provide answers in all official languages"
response_text= detect_and_respond(user_query)
display(Markdown(f"**Sg Helper Bot:** {response_text}"))

**Sg Helper Bot:** Okay, here is the information you requested:

**Trend of Housing Prices in Singapore (Answer in English):**

Overall, the trend for housing prices in Singapore (both public HDB flats and private properties) has been **upwards** over the past several years, particularly since the start of the COVID-19 pandemic. Demand has been strong, fueled by various economic factors and aspirations for homeownership.

However, more recently (leading into 2024), there are signs of **moderation** in the rate of price increases. This is largely attributed to:

1.  **Government Cooling Measures:** Successive rounds of property cooling measures, including increased Additional Buyer's Stamp Duty (ABSD) rates, have aimed to temper demand, especially for investment properties.
2.  **Higher Interest Rates:** Rising global interest rates have increased mortgage costs, potentially dampening affordability and buyer sentiment.
3.  **Increased Supply:** Efforts to increase the supply of new HDB flats (BTOs) and private housing aim to better match demand over the medium term.

**In summary:** While the long-term trend has been significant growth, the recent trend shows a **slowing pace of price appreciation** across both HDB resale and private property markets, though prices generally remain high. The rental market also saw sharp increases but is showing signs of stabilizing or slight easing.

---

**Translation of your request ("Get me a trend of the housing prices in Singapore") into Singapore's official languages:**

*   **English:** Get me a trend of the housing prices in Singapore.
*   **Malay (Bahasa Melayu):** Berikan saya trend harga perumahan di Singapura.
*   **Mandarin Chinese (华语):** 请给我新加坡的房价趋势。 (Qǐng gěi wǒ Xīnjiāpō de fángjià qūshì.)
*   **Tamil (தமிழ்):** சிங்கப்பூரில் வீட்டு விலைகளின் போக்கை எனக்குக் கொடுங்கள். (Singappūril veettu vilaigalin pōkkai enakkuk kodungal.)

## Structured output in JSON 

**Let us say that we want to send the response back as a JSON , for instance as a response to an API call, use this structured output feature**

In [15]:
from google.genai import types
from IPython.display import JSON

model_structured = GenerativeModel(
    'gemini-2.5-pro-exp-03-25',
    generation_config={"response_mime_type": "application/json"}
)

def multilingual_helper_structured(user_input, target_language=None):
    if target_language:
        prompt = f"Translate the following and answer in {target_language}. Provide the response in JSON: {user_input}"
    else:
        prompt = f"Detect the language and answer in the same language. Provide the response in JSON: {user_input}"
    response = model.generate_content(prompt)
    return response.text

def detect_and_respond_strcutured(user_input):
    lang = detect(user_input)
    return multilingual_helper_structured(user_input, target_language=lang)


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:623: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [16]:
user_input="Get me a trend of the housing prices in Singapore. Provide answers in all official languages "
response_text=detect_and_respond_strcutured(user_input)
response_text

'```json\n{\n  "housing_price_trend_singapore": {\n    "en": "Overall, Singapore\'s housing prices have shown a long-term upward trend for both private properties and HDB resale flats. While there have been periods of fluctuation and moderation, often influenced by government cooling measures and economic conditions, the market has generally demonstrated resilience and consistent growth over the decades. Recent years have continued to see price increases, although the pace may moderate due to rising interest rates and recent policy adjustments.",\n    "ms": "Secara keseluruhannya, harga perumahan di Singapura telah menunjukkan trend menaik jangka panjang untuk kedua-dua hartanah swasta dan flat jualan semula HDB. Walaupun terdapat tempoh turun naik dan penyederhanaan, yang sering dipengaruhi oleh langkah-langkah penyejukan kerajaan dan keadaan ekonomi, pasaran secara amnya telah menunjukkan daya tahan dan pertumbuhan yang konsisten selama beberapa dekad. Tahun-tahun kebelakangan ini te

In [17]:
print(response_text)

```json
{
  "housing_price_trend_singapore": {
    "en": "Overall, Singapore's housing prices have shown a long-term upward trend for both private properties and HDB resale flats. While there have been periods of fluctuation and moderation, often influenced by government cooling measures and economic conditions, the market has generally demonstrated resilience and consistent growth over the decades. Recent years have continued to see price increases, although the pace may moderate due to rising interest rates and recent policy adjustments.",
    "ms": "Secara keseluruhannya, harga perumahan di Singapura telah menunjukkan trend menaik jangka panjang untuk kedua-dua hartanah swasta dan flat jualan semula HDB. Walaupun terdapat tempoh turun naik dan penyederhanaan, yang sering dipengaruhi oleh langkah-langkah penyejukan kerajaan dan keadaan ekonomi, pasaran secara amnya telah menunjukkan daya tahan dan pertumbuhan yang konsisten selama beberapa dekad. Tahun-tahun kebelakangan ini terus me

## 6. Enable Grounding with Google Search 

**We use google-genai v1.7.0 for this section** 

In [18]:
#google.generativeai' has no attribute 'Tool' Need to explore more for grounding
#use google-genai

# Uninstall packages from Kaggle base image that are not needed.
!pip uninstall -qy jupyterlab jupyterlab-lsp
# Install the google-genai SDK for this codelab.
!pip install -qU 'google-genai==1.7.0'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 6.1 MB/s eta 0:00:00


In [19]:
#enable grounding 
from google import genai
from google.genai import types

from kaggle_secrets import UserSecretsClient
from IPython.display import Markdown, HTML, display

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)









## 7. Retry mechanism - to use when client retries if it hits quota limits 

In [20]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)


## 8. multilingual helper with grounding enabled 

*This takes a model_to_use param so that different models can be called* 

In [21]:
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
)


def multilingual_helper_with_grounding(user_input, target_language=None,model_to_use='gemini-2.0-flash'):
    if target_language:
        prompt = f"Translate the following and answer in {target_language}: {user_input}"
    else:
        prompt = f"Detect the language and answer in the same language: {user_input}"
    #response = model_with_grounding.generate_content(prompt,grounding_source=genai.GroundingSource.GOOGLE_SEARCH)
    response = client.models.generate_content(
        model=model_to_use,
        contents=prompt,
        config=config_with_search,
    )
    return response.text

def detect_and_respond_with_grounding(user_input,model_to_use='gemini-2.0-flash'):
    lang = detect(user_input)
    return multilingual_helper_with_grounding(user_input, target_language=lang,model_to_use=model_to_use)



## 9. Query using gemini-2.0-flash (default model in helper fn) and get Multilingiual response in all Sg official languages 

In [22]:
user_query="Get me a trend of the housing prices in Singapore. Provide answers in all official languages"
response_text= detect_and_respond_with_grounding(user_query) #use gemini_2.0-flash
display(Markdown(f"**Sg Helper Bot:** {response_text}"))

**Sg Helper Bot:** Here's the translation and analysis of housing price trends in Singapore, provided in English, Mandarin, Malay, and Tamil:

**English:**

You're asking for a trend analysis of housing prices in Singapore. To provide a comprehensive overview, I need to consider several factors and data points. Here's a general outline of what the trend looks like:

*   **Overall Trend:** Singapore's property market is known for its stability, but prices do fluctuate based on economic conditions, government policies, and global events.
*   **Recent Trends:** (Based on current data up to April 20, 2025) There has been a general upward trend in housing prices in recent years, particularly after the pandemic. However, there have been some cooling measures implemented by the government to moderate price increases. These measures include increased stamp duties and stricter loan-to-value ratios.
*   **Types of Housing:** Trends can vary significantly between different types of housing:
    *   **HDB (Public Housing):** HDB resale prices have generally been increasing, driven by demand and relatively limited supply. Government policies play a significant role in this market.
    *   **Private Condominiums:** Private condo prices have also been on the rise, influenced by factors like foreign investment, economic growth, and interest rates.
    *   **Landed Properties:** Landed properties (houses with land) tend to be the most expensive and can see significant price fluctuations based on demand from high-net-worth individuals.

To get the most up-to-date and precise information, I would recommend consulting these sources:

*   **Real Estate Portals:** PropertyGuru, 99.co, and SRX are popular portals that track property prices in Singapore.
*   **Government Agencies:** The Urban Redevelopment Authority (URA) and the Housing Development Board (HDB) release official data and reports on property market trends.
*   **News Outlets:** The Straits Times and Business Times provide regular updates and analysis of the Singapore property market.

Now, let's provide the same information in the other official languages.

Okay, I have updated information regarding the housing price trends in Singapore up to April 2025. Here's the breakdown in all four official languages:

**中文 (Chinese/Mandarin):**

您要求对新加坡的房价趋势进行分析。为了提供全面的概述，我需要考虑几个因素和数据点。以下是总体趋势的样子：

*   **总体趋势：** 新加坡的房地产市场以其稳定性而闻名，但价格会根据经济状况、政府政策和全球事件而波动。
*   **近期趋势：**（基于截至2025年4月20日的最新数据）近年来，房价总体呈上涨趋势，尤其是在疫情之后。但是，政府已经实施了一些降温措施来缓和价格上涨。这些措施包括提高印花税和更严格的贷款价值比率。
*   **房屋类型：** 不同类型的房屋的趋势可能差异很大：
    *   **组屋（公共住房）：** 组屋转售价格普遍上涨，主要受需求和供应相对有限的推动。政府政策在这个市场中起着重要作用。
    *   **私人公寓：** 私人公寓的价格也在上涨，受到外国投资、经济增长和利率等因素的影响。
    *   **有地房产：** 有地房产（带土地的房屋）往往是最昂贵的，并且可能因高净值人士的需求而出现 значительные价格波动。

为了获得最新和最准确的信息，我建议咨询以下来源：

*   **房地产门户网站：** PropertyGuru、99.co 和 SRX 是跟踪新加坡房地产价格的热门门户网站。
*   **政府机构：** 市区重建局（URA）和建屋发展局（HDB）发布有关房地产市场趋势的官方数据和报告。
*   **新闻媒体：** 《海峡时报》和《商业时报》定期提供有关新加坡房地产市场的更新和分析。

根据Trading Economics的分析师预期，新加坡的房价指数预计在本季度末将达到207.00点。从长期来看，预计到2026年，新加坡住宅房地产价格指数将达到241.00点左右。

**Melayu (Malay):**

Anda meminta analisis trend harga rumah di Singapura. Untuk memberikan gambaran yang komprehensif, saya perlu mempertimbangkan beberapa faktor dan titik data. Berikut adalah gambaran keseluruhan tentang rupa trend tersebut:

*   **Trend Keseluruhan:** Pasaran hartanah Singapura terkenal dengan kestabilannya, tetapi harga turun naik berdasarkan keadaan ekonomi, dasar kerajaan, dan peristiwa global.
*   **Trend Terkini:** (Berdasarkan data terkini sehingga 20 April 2025) Terdapat trend menaik umum dalam harga rumah sejak beberapa tahun kebelakangan ini, terutamanya selepas pandemik. Walau bagaimanapun, terdapat beberapa langkah penyejukan yang dilaksanakan oleh kerajaan untuk mengurangkan kenaikan harga. Langkah-langkah ini termasuk peningkatan duti setem dan nisbah pinjaman kepada nilai yang lebih ketat.
*   **Jenis Perumahan:** Trend boleh berbeza dengan ketara antara jenis perumahan yang berbeza:
    *   **HDB (Perumahan Awam):** Harga jualan semula HDB secara amnya meningkat, didorong oleh permintaan dan bekalan yang agak terhad. Dasar kerajaan memainkan peranan penting dalam pasaran ini.
    *   **Kondominium Persendirian:** Harga kondominium persendirian juga meningkat, dipengaruhi oleh faktor seperti pelaburan asing, pertumbuhan ekonomi, dan kadar faedah.
    *   **Hartanah BerTanah:** Hartanah bertanah (rumah dengan tanah) cenderung menjadi yang paling mahal dan boleh melihat turun naik harga yang ketara berdasarkan permintaan daripada individu bernilai bersih tinggi.

Untuk mendapatkan maklumat yang paling terkini dan tepat, saya mengesyorkan untuk merujuk sumber-sumber ini:

*   **Portal Harta Tanah:** PropertyGuru, 99.co, dan SRX ialah portal popular yang menjejaki harga hartanah di Singapura.
*   **Agensi Kerajaan:** Lembaga Pembangunan Bandar (URA) dan Lembaga Perumahan dan Pembangunan (HDB) mengeluarkan data dan laporan rasmi mengenai trend pasaran hartanah.
*   **Kedai Berita:** The Straits Times dan Business Times menyediakan kemas kini dan analisis tetap mengenai pasaran hartanah Singapura.

Menurut jangkaan penganalisis Trading Economics, Indeks Harga Rumah di Singapura dijangka berada pada 207.00 mata menjelang akhir suku tahun ini. Dalam jangka panjang, Indeks Harga Hartanah Kediaman Singapura diunjurkan untuk trend sekitar 241.00 mata pada tahun 2026.

**தமிழ் (Tamil):**

சிங்கப்பூரில் வீட்டு விலைகளின் போக்கு பற்றிய ஆய்வை நீங்கள் கேட்கிறீர்கள். ஒரு விரிவான கண்ணோட்டத்தை வழங்க, நான் பல காரணிகளையும் தரவு புள்ளிகளையும் கருத்தில் கொள்ள வேண்டும். போக்கு எப்படி இருக்கிறது என்பதற்கான ஒரு பொதுவான சுருக்கம் இங்கே:

*   **ஒட்டுமொத்த போக்கு:** சிங்கப்பூரின் சொத்துச் சந்தை அதன் ஸ்திரத்தன்மைக்கு பெயர் பெற்றது, ஆனால் பொருளாதார நிலைமைகள், அரசாங்கக் கொள்கைகள் மற்றும் உலகளாவிய நிகழ்வுகளின் அடிப்படையில் விலைகள் மாறுகின்றன.
*   **சமீபத்திய போக்குகள்:** (ஏப்ரல் 20, 2025 வரையிலான தற்போதைய தரவுகளின்படி) சமீபத்திய ஆண்டுகளில் வீட்டு விலைகளில் பொதுவான உயர்வு காணப்படுகிறது, குறிப்பாக தொற்றுக்குப் பிறகு. இருப்பினும், விலை உயர்வை மிதப்படுத்த அரசாங்கம் சில குளிர்விக்கும் நடவடிக்கைகளை எடுத்துள்ளது. இந்த நடவடிக்கைகளில் அதிகரித்த முத்திரை வரி மற்றும் கடுமையான கடன்-மதிப்பு விகிதங்கள் ஆகியவை அடங்கும்.
*   **வீட்டு வகைகள்:** வீட்டு வகைகளுக்கு இடையே போக்குகள் கணிசமாக வேறுபடலாம்:
    *   **HDB (பொது வீட்டுவசதி):** HDB மறுவிற்பனை விலைகள் பொதுவாக அதிகரித்து வருகின்றன, இது தேவை மற்றும் ஒப்பீட்டளவில் வரையறுக்கப்பட்ட விநியோகத்தால் இயக்கப்படுகிறது. இந்த சந்தையில் அரசாங்கக் கொள்கைகள் ஒரு முக்கிய பங்கு வகிக்கின்றன.
    *   **தனியார் காண்டோமினியங்கள்:** வெளிநாட்டு முதலீடு, பொருளாதார வளர்ச்சி மற்றும் வட்டி விகிதங்கள் போன்ற காரணிகளால் தனியார் காண்டோமினியங்களின் விலையும் அதிகரித்து வருகிறது.
    *   **நிலையான சொத்துக்கள்:** நிலையான சொத்துக்கள் (நிலத்துடன் கூடிய வீடுகள்) மிகவும் விலை உயர்ந்ததாக இருக்கும் மற்றும் அதிக நிகர மதிப்புள்ள நபர்களிடமிருந்து வரும் தேவையின் அடிப்படையில் குறிப்பிடத்தக்க விலை ஏற்ற இறக்கங்களைக் காணலாம்.

மிகவும் சமீபத்திய மற்றும் துல்லியமான தகவல்களைப் பெற, இந்த ஆதாரங்களைக் கலந்தாலோசிக்குமாறு பரிந்துரைக்கிறேன்:

*   **சொத்து இணையதளங்கள்:** PropertyGuru, 99.co மற்றும் SRX ஆகியவை சிங்கப்பூரில் உள்ள சொத்து விலைகளைக் கண்காணிக்கும் பிரபலமான இணையதளங்கள்.
*   **அரசாங்க முகவர் நிலையங்கள்:** நகர மறுவடிவமைப்பு ஆணையம் (URA) மற்றும் வீட்டுவசதி மேம்பாட்டு வாரியம் (HDB) சொத்துச் சந்தைப் போக்குகள் குறித்த அதிகாரப்பூர்வ தரவு மற்றும் அறிக்கைகளை வெளியிடுகின்றன.
*   **செய்தி நிறுவனங்கள்:** ஸ்ட்ரெயிட்ஸ் டைம்ஸ் மற்றும் பிசினஸ் டைம்ஸ் ஆகியவை சிங்கப்பூர் சொத்துச் சந்தையின் வழக்கமான புதுப்பிப்புகள் மற்றும் பகுப்பாய்வுகளை வழங்குகின்றன.

Trading Economics ஆய்வாளர்களின் எதிர்பார்ப்புகளின்படி, சிங்கப்பூரின் வீட்டு விலை குறியீடு இந்த காலாண்டின் இறுதியில் 207.00 புள்ளிகளாக இருக்கும் என்று எதிர்பார்க்கப்படுகிறது. நீண்ட கால அடிப்படையில், சிங்கப்பூர் குடியிருப்பு சொத்து விலை குறியீடு 2026 ஆம் ஆண்டில் சுமார் 241.00 புள்ளிகளாக இருக்கும் என்று கணிக்கப்பட்டுள்ளது.


## 10. get multilingual response for the same query using gemini-2.5-pro-exp-03-25 

In [23]:
user_query="Get me a trend of the housing prices in Singapore. Provide answers in all official languages"
response_text= detect_and_respond_with_grounding(user_query,'gemini-2.5-pro-exp-03-25') #use gemini-2.5-pro-exp-03-25
display(Markdown(f"**Sg Helper Bot:** {response_text}"))

**Sg Helper Bot:** Here is a summary of the housing price trends in Singapore, presented in English, Malay, Mandarin Chinese, and Tamil:

**English**

Overall, the Singapore housing market, encompassing both public (HDB) and private residential properties, continues to see price increases, but the pace of growth has shown signs of moderation in late 2024 and early 2025 compared to previous years. This moderation is attributed to factors like increased housing supply, government cooling measures (including higher Additional Buyer's Stamp Duty (ABSD) rates, particularly for foreigners), and a high-interest-rate environment.

*   **Private Residential Market:**
    *   Prices for private residential properties (condos and landed houses) continued to rise overall in 2024, though growth slowed compared to 2022 and 2023. The overall private home price index grew 3.9% in 2024, compared to 6.8% in 2023 and 8.6% in 2022.
    *   Flash estimates for Q1 2025 showed a further slowdown, with prices increasing by 0.6%, down from 2.3% in Q4 2024.
    *   Transaction volumes dropped in Q1 2025 compared to the previous quarter. However, Q3 2024 saw a slight recovery in sales after a low in H1 2024.
    *   New home sales saw a robust quarterly increase in Q3 2024, driven by launches in the Rest of Central Region (RCR) and Outside Central Region (OCR). However, new sales were lower year-on-year.
    *   Forecasts for 2025 suggest moderate price growth, potentially between 3% and 5%, supported by limited new supply, expected interest rate cuts, and resilient demand, particularly from HDB upgraders.
    *   The rental market saw a slowdown in 2024 due to increased supply from newly completed units, but is expected to stabilize or see moderate growth in 2025 as new completions decrease.

*   **HDB Resale Market:**
    *   HDB resale prices saw significant growth in 2024, rising 9.6%, which was faster than the 4.9% increase in 2023. Prices have risen for 20 consecutive quarters.
    *   However, price growth slowed in Q4 2024 (2.5% increase) and further moderated in Q1 2025, rising by 1.5%, the slowest growth in five quarters.
    *   Resale transaction volume dropped in Q1 2025 compared to the same period last year.
    *   The number of million-dollar HDB flat transactions continued to rise, reaching a record high in February 2025 despite lower overall transaction volumes.
    *   Factors influencing the HDB market include sustained BTO flat supply, government cooling measures (like tightened LTV ratios for HDB loans), and potentially buyer resistance to higher prices. Year-on-year price increases remained strong as of March 2025 (9.5% compared to March 2024).

In summary, while housing prices in Singapore are still trending upwards, the rate of increase has slowed, suggesting a period of stabilization influenced by increased supply and policy measures.

---

**Melayu (Malay)**

Secara keseluruhan, pasaran perumahan Singapura, merangkumi kedua-dua hartanah kediaman awam (HDB) dan swasta, terus menyaksikan kenaikan harga, tetapi kadar pertumbuhan menunjukkan tanda-tanda kesederhanaan pada akhir 2024 dan awal 2025 berbanding tahun-tahun sebelumnya. Kesederhanaan ini disebabkan oleh faktor-faktor seperti peningkatan bekalan perumahan, langkah-langkah penyejukan kerajaan (termasuk kadar Duti Setem Pembeli Tambahan (ABSD) yang lebih tinggi, terutamanya untuk warga asing), dan persekitaran kadar faedah yang tinggi.

*   **Pasaran Kediaman Swasta:**
    *   Harga untuk hartanah kediaman swasta (kondominium dan rumah bertanah) terus meningkat secara keseluruhan pada tahun 2024, walaupun pertumbuhannya lebih perlahan berbanding tahun 2022 dan 2023. Indeks harga rumah persendirian keseluruhan meningkat 3.9% pada tahun 2024, berbanding 6.8% pada tahun 2023 dan 8.6% pada tahun 2022.
    *   Anggaran awal untuk Q1 2025 menunjukkan kelembapan selanjutnya, dengan harga meningkat sebanyak 0.6%, turun daripada 2.3% pada Q4 2024.
    *   Jumlah transaksi menurun pada Q1 2025 berbanding suku sebelumnya. Walau bagaimanapun, Q3 2024 menyaksikan sedikit pemulihan dalam jualan selepas paras rendah pada H1 2024.
    *   Jualan rumah baharu menyaksikan peningkatan suku tahunan yang kukuh pada Q3 2024, didorong oleh pelancaran di Rest of Central Region (RCR) dan Outside Central Region (OCR). Walau bagaimanapun, jualan baharu adalah lebih rendah dari tahun ke tahun.
    *   Ramalan untuk tahun 2025 mencadangkan pertumbuhan harga yang sederhana, berkemungkinan antara 3% dan 5%, disokong oleh bekalan baharu yang terhad, jangkaan pemotongan kadar faedah, dan permintaan yang berdaya tahan, terutamanya daripada penaik taraf HDB.
    *   Pasaran sewa menyaksikan kelembapan pada tahun 2024 disebabkan oleh peningkatan bekalan daripada unit yang baru siap, tetapi dijangka stabil atau menyaksikan pertumbuhan sederhana pada tahun 2025 apabila penyiapan baharu berkurangan.

*   **Pasaran Jualan Semula HDB:**
    *   Harga jualan semula HDB menyaksikan pertumbuhan yang ketara pada tahun 2024, meningkat 9.6%, lebih cepat daripada kenaikan 4.9% pada tahun 2023. Harga telah meningkat selama 20 suku berturut-turut.
    *   Walau bagaimanapun, pertumbuhan harga menjadi perlahan pada Q4 2024 (kenaikan 2.5%) dan terus sederhana pada Q1 2025, meningkat sebanyak 1.5%, pertumbuhan paling perlahan dalam lima suku terakhir.
    *   Jumlah transaksi jualan semula menurun pada Q1 2025 berbanding tempoh yang sama tahun lepas.
    *   Bilangan transaksi flat HDB bernilai sejuta dolar terus meningkat, mencapai rekod tertinggi pada Februari 2025 walaupun jumlah transaksi keseluruhan lebih rendah.
    *   Faktor-faktor yang mempengaruhi pasaran HDB termasuk bekalan flat BTO yang berterusan, langkah-langkah penyejukan kerajaan (seperti nisbah LTV yang lebih ketat untuk pinjaman HDB), dan kemungkinan penolakan pembeli terhadap harga yang lebih tinggi. Kenaikan harga dari tahun ke tahun kekal kukuh setakat Mac 2025 (9.5% berbanding Mac 2024).

Ringkasnya, walaupun harga perumahan di Singapura masih menunjukkan arah aliran menaik, kadar kenaikan telah perlahan, mencadangkan tempoh penstabilan yang dipengaruhi oleh peningkatan bekalan dan langkah-langkah dasar.

---

**华语 (Mandarin Chinese)**

总体而言，新加坡的住宅市场（包括公共组屋 HDB 和私人住宅）价格持续上涨，但与前几年相比，2024年底和2025年初的增长速度已显示出放缓迹象。这种放缓归因于住房供应增加、政府降温措施（包括提高额外买方印花税 ABSD 税率，特别是针对外国人）以及高利率环境等因素。

*   **私人住宅市场:**
    *   2024年，私人住宅（公寓和有地住宅）的价格总体持续上涨，但增速较 2022 年和 2023 年有所放缓。 整体私人住宅价格指数在 2024 年上涨了 3.9%，而 2023 年为 6.8%，2022 年为 8.6%。
    *   2025 年第一季度的初步估计显示增长进一步放缓，价格上涨 0.6%，低于 2024 年第四季度的 2.3%。
    *   2025 年第一季度的交易量较上一季度有所下降。 然而，在 2024 年上半年销售低迷之后，2024 年第三季度销售略有回升。
    *   在其他中央区（RCR）和中央区外（OCR）推出新楼盘的推动下，2024 年第三季度新房销售额实现了强劲的季度增长。 然而，新销售额同比下降。
    *   预测显示，2025 年房价将温和增长，可能在 3% 至 5% 之间，这得益于有限的新供应量、预期的降息以及有韧性的需求（特别是来自组屋升级者的需求）。
    *   由于新竣工单位供应增加，2024 年租赁市场放缓，但随着新竣工量减少，预计 2025 年将趋于稳定或温和增长。

*   **组屋转售市场:**
    *   2024 年组屋转售价格大幅上涨 9.6%，快于 2023 年 4.9% 的涨幅。 价格已连续 20 个季度上涨。
    *   然而，价格增长在 2024 年第四季度放缓（上涨 2.5%），并在 2025 年第一季度进一步放缓，上涨 1.5%，为五个季度以来的最低增幅。
    *   2025 年第一季度的转售交易量与去年同期相比有所下降。
    *   尽管整体交易量下降，但百万元组屋单位的交易数量持续增加，并在 2025 年 2 月创下历史新高。
    *   影响组屋市场的因素包括持续的预购组屋（BTO）供应、政府的降温措施（如收紧组屋贷款的贷款与价值比率 LTV），以及买家可能对高房价产生抵触情绪。 截至 2025 年 3 月，同比价格涨幅依然强劲（与 2024 年 3 月相比上涨 9.5%）。

总而言之，虽然新加坡的房价仍呈上涨趋势，但涨幅已经放缓，表明在供应增加和政策措施的影响下，市场进入了一个稳定期。

---

**தமிழ் (Tamil)**

ஒட்டுமொத்தமாக, சிங்கப்பூர் வீட்டமைப்பு சந்தை, பொது (HDB) மற்றும் தனியார் குடியிருப்புகள் இரண்டையும் உள்ளடக்கியது, தொடர்ந்து விலையேற்றத்தைக் காண்கிறது, ஆனால் முந்தைய ஆண்டுகளுடன் ஒப்பிடும்போது 2024 இன் பிற்பகுதியிலும் 2025 இன் முற்பகுதியிலும் வளர்ச்சி வேகம் மிதமானதற்கான அறிகுறிகளைக் காட்டுகிறது. இந்த மிதப்படுத்தலுக்கு வீட்டு விநியோகம் அதிகரிப்பு, அரசாங்கத்தின் குளிர்விப்பு நடவடிக்கைகள் (குறிப்பாக வெளிநாட்டினருக்கான கூடுதல் வாங்குபவர் முத்திரை வரி (ABSD) விகிதங்கள் உட்பட) மற்றும் உயர் வட்டி விகித சூழல் போன்ற காரணிகள் காரணமாகின்றன.

*   **தனியார் குடியிருப்பு சந்தை:**
    *   தனியார் குடியிருப்புகளின் (காண்டோக்கள் மற்றும் நிலம்சார் வீடுகள்) விலைகள் 2024 இல் ஒட்டுமொத்தமாக தொடர்ந்து உயர்ந்தன, இருப்பினும் 2022 மற்றும் 2023 உடன் ஒப்பிடும்போது வளர்ச்சி குறைந்துள்ளது. ஒட்டுமொத்த தனியார் வீட்டு விலை குறியீடு 2024 இல் 3.9% ஆக வளர்ந்தது, இது 2023 இல் 6.8% ஆகவும் 2022 இல் 8.6% ஆகவும் இருந்தது.
    *   2025 முதல் காலாண்டிற்கான ஃபிளாஷ் மதிப்பீடுகள் மேலும் ஒரு மந்தநிலையைக் காட்டின, விலைகள் 0.6% அதிகரித்தன, இது 2024 நான்காம் காலாண்டில் 2.3% ஆக இருந்தது.
    *   2025 முதல் காலாண்டில் பரிவர்த்தனை அளவு முந்தைய காலாண்டுடன் ஒப்பிடும்போது குறைந்துள்ளது. இருப்பினும், 2024 முதல் பாதியில் ஏற்பட்ட குறைவுக்குப் பிறகு, 2024 மூன்றாம் காலாண்டில் விற்பனையில் சிறிய மீட்சி காணப்பட்டது.
    *   மத்திய பிராந்தியத்தின் மற்ற பகுதி (RCR) மற்றும் மத்திய பிராந்தியத்திற்கு வெளியே (OCR) தொடங்கப்பட்டவற்றால் இயக்கப்பட்டு, 2024 மூன்றாம் காலாண்டில் புதிய வீட்டு விற்பனைகள் வலுவான காலாண்டு அதிகரிப்பைக் கண்டன. இருப்பினும், புதிய விற்பனைகள் ஆண்டுக்கு ஆண்டு குறைவாக இருந்தன.
    *   2025 ஆம் ஆண்டுக்கான முன்னறிவிப்புகள் மிதமான விலை வளர்ச்சியைக் குறிக்கின்றன, இது 3% முதல் 5% வரை இருக்கலாம், இது வரையறுக்கப்பட்ட புதிய விநியோகம், எதிர்பார்க்கப்படும் வட்டி விகிதக் குறைப்புகள் மற்றும் நெகிழ்வான தேவை, குறிப்பாக HDB மேம்படுத்துபவர்களிடமிருந்து ஆதரவைப் பெறுகிறது.
    *   புதிதாக முடிக்கப்பட்ட அலகுகளிலிருந்து அதிகரித்த விநியோகம் காரணமாக 2024 இல் வாடகைச் சந்தை மந்தநிலையைக் கண்டது, ஆனால் புதிய நிறைவுகள் குறைவதால் 2025 இல் நிலைப்படுத்தப்படும் அல்லது மிதமான வளர்ச்சியைக் காணும் என்று எதிர்பார்க்கப்படுகிறது.

*   **HDB மறுவிற்பனை சந்தை:**
    *   HDB மறுவிற்பனை விலைகள் 2024 இல் குறிப்பிடத்தக்க வளர்ச்சியைக் கண்டன, 9.6% உயர்ந்தன, இது 2023 இல் பதிவு செய்யப்பட்ட 4.9% அதிகரிப்பை விட வேகமானது. விலைகள் தொடர்ந்து 20 காலாண்டுகளாக உயர்ந்துள்ளன.
    *   இருப்பினும், விலை வளர்ச்சி 2024 நான்காம் காலாண்டில் (2.5% அதிகரிப்பு) குறைந்து, 2025 முதல் காலாண்டில் மேலும் மிதப்படுத்தப்பட்டது, 1.5% உயர்ந்தது, இது ஐந்து காலாண்டுகளில் மிக மெதுவான வளர்ச்சியாகும்.
    *   2025 முதல் காலாண்டில் மறுவிற்பனை பரிவர்த்தனை அளவு கடந்த ஆண்டு இதே காலத்துடன் ஒப்பிடும்போது குறைந்துள்ளது.
    *   ஒட்டுமொத்த பரிவர்த்தனை அளவுகள் குறைவாக இருந்தபோதிலும், மில்லியன் டாலர் HDB அடுக்குமாடி பரிவர்த்தனைகளின் எண்ணிக்கை தொடர்ந்து உயர்ந்து, பிப்ரவரி 2025 இல் ஒரு புதிய சாதனையை எட்டியது.
    *   HDB சந்தையைப் பாதிக்கும் காரணிகள், நீடித்த BTO பிளாட் விநியோகம், அரசாங்க குளிர்விப்பு நடவடிக்கைகள் (HDB கடன்களுக்கான இறுக்கமான LTV விகிதங்கள் போன்றவை), மற்றும் அதிக விலைகளுக்கு வாங்குபவர் எதிர்ப்பு ஆகியவை அடங்கும். மார்ச் 2025 நிலவரப்படி ஆண்டுக்கு ஆண்டு விலை உயர்வு வலுவாக இருந்தது (மார்ச் 2024 உடன் ஒப்பிடும்போது 9.5%).

சுருக்கமாக, சிங்கப்பூரில் வீட்டு விலைகள் இன்னும் மேல்நோக்கிய போக்கில் இருந்தாலும், அதிகரிப்பு விகிதம் குறைந்துள்ளது, இது அதிகரித்த விநியோகம் மற்றும் கொள்கை நடவடிக்கைகளால் பாதிக்கப்பட்ட ஒரு நிலைப்படுத்தல் காலத்தைக் குறிக்கிறது.

## Comparing responses from the two models using grounding with Google Search . 
## While the response is better formatted with gemini-2.5-pro-exp-03-25 using Google search grounding , its reference time frame is the first quarter of 2024!!
## Gemini-Flash-2.0 seems to have a more recent reference point , the answers were more concise. 
## Need to explore this aspect further 

****Extending the Bot:
You can add Data.gov.sg or other APIs as tools for even richer, hyperlocal answers.
Evaluation:
Try queries like “Where can I get halal food near Bugis?”, “List clinics open near Kembangan”****